In [1]:
#!pip install pretty_midi

In [2]:
import pretty_midi as pm
import numpy as np

In [3]:
test_midi = pm.PrettyMIDI('Dataset/1.mid')

In [4]:
test_midi

In [5]:
test_midi.instruments

[Instrument(program=0, is_drum=False, name="Nexus 1")]

In [6]:
test_midi.instruments[0].notes

[Note(start=0.000000, end=0.230769, pitch=76, velocity=100),
 Note(start=0.346153, end=0.576922, pitch=76, velocity=100),
 Note(start=0.692307, end=0.923076, pitch=76, velocity=100),
 Note(start=1.038460, end=1.269229, pitch=76, velocity=100),
 Note(start=1.384614, end=1.615383, pitch=74, velocity=100),
 Note(start=1.615383, end=1.846152, pitch=72, velocity=100),
 Note(start=1.846152, end=2.076921, pitch=79, velocity=100),
 Note(start=2.192305, end=2.423074, pitch=79, velocity=100),
 Note(start=2.538459, end=2.769228, pitch=76, velocity=100),
 Note(start=2.884612, end=3.115381, pitch=76, velocity=100),
 Note(start=3.230766, end=3.461535, pitch=74, velocity=100),
 Note(start=3.461535, end=3.692304, pitch=72, velocity=100),
 Note(start=3.692304, end=3.923073, pitch=79, velocity=100),
 Note(start=4.038457, end=4.269226, pitch=79, velocity=100),
 Note(start=4.384611, end=4.615380, pitch=76, velocity=100),
 Note(start=4.730764, end=4.961533, pitch=76, velocity=100),
 Note(start=5.076918, en

In [7]:
test_midi.instruments[0].notes[0].pitch

76

In [3]:
def midi_to_list(midi):
    pitch_list = []
    for note in midi.instruments[0].notes:
        pitch_list.append(note.pitch)
    return pitch_list

def list_shift(pitch_list, lowest, highest):
    pitch_list_new = []
    for note in pitch_list:
        if(note > highest):
            print('note with pitch ' + str(note) + 'spotted')
        pitch_list_new.append(note - lowest)
    return pitch_list_new

In [9]:
list_shift(midi_to_list(test_midi),69,81)#0 - A5, 12 - A6

[7, 7, 7, 7, 5, 3, 10, 10, 7, 7, 5, 3, 10, 10, 7, 7, 5, 3, 7, 7, 5, 5, 3, 2]

In [4]:
notes_4_first_opt = {0: 0, 2: 1, 3:2, 5:3, 7:4, 8:5, 10:6, 12:7}

def first_opt(pitch_list_new):
    tmp = []
    for note in pitch_list_new:
        tmp.append(notes_4_first_opt[note])
    return tmp

In [11]:
first_opt(list_shift(midi_to_list(test_midi),69,81))

[4, 4, 4, 4, 3, 2, 6, 6, 4, 4, 3, 2, 6, 6, 4, 4, 3, 2, 4, 4, 3, 3, 2, 1]

# Загрузка тренировочного датасета

In [79]:
data_full = np.zeros((34,24))

for num in range(0,21):
    tmp_midi = pm.PrettyMIDI('Dataset/'+ str(num+1) + '.mid')
    tmp_midi_edited = first_opt(list_shift(midi_to_list(tmp_midi),69,81))
    data_full[num] = tmp_midi_edited

In [80]:
#теперь нужно преобразовать датасет для нейросети

data_4_nn = np.zeros((34, 24, 8))

for melody in range(0, 34):
    for melody_note in range(0, 24):
            data_4_nn[melody][melody_note][int(data_full[melody][melody_note])] = 1

In [81]:
#а потом вот в такое чудо

data_4_nn_fin = np.zeros((34,192))

for melody in range(34):
    for melody_note in range(24):
        for note in range(0,8):
            data_4_nn_fin[melody][8*melody_note + note] = data_4_nn[melody][melody_note][note]

In [82]:
#Создание последовательностей

X = []
y = []

for melody in range(34):
    for melody_note in range(19):
        X.append([data_4_nn[melody][melody_note],data_4_nn[melody][melody_note+1],data_4_nn[melody][melody_note+2],data_4_nn[melody][melody_note+3],data_4_nn[melody][melody_note+4]])
        y.append(data_4_nn[melody][melody_note+5])

In [83]:
X = np.array(X)
y = np.array(y)

In [84]:
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, LSTM
#from keras.utils import np_utils
#from keras.callbacks import ModelCheckpoint

In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

In [86]:
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [87]:
model.compile(loss='categorical_crossentropy', optimizer='adam') 

In [88]:
filepath = "model_1_weights_saved.hdf5" 
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min') 
desired_callbacks = [checkpoint] 

In [89]:
model.fit(X, y, epochs = 150, batch_size = 64, callbacks=desired_callbacks)

Epoch 1/150
11/11 [==============================] - ETA: 0s - loss: 1.7066
Epoch 00001: loss improved from inf to 1.70656, saving model to model_1_weights_saved.hdf5
11/11 [==============================] - 1s 124ms/step - loss: 1.7066
Epoch 2/150
11/11 [==============================] - ETA: 0s - loss: 1.2987
Epoch 00002: loss improved from 1.70656 to 1.29866, saving model to model_1_weights_saved.hdf5
11/11 [==============================] - 1s 119ms/step - loss: 1.2987
Epoch 3/150
11/11 [==============================] - ETA: 0s - loss: 1.1581
Epoch 00003: loss improved from 1.29866 to 1.15807, saving model to model_1_weights_saved.hdf5
11/11 [==============================] - 1s 115ms/step - loss: 1.1581
Epoch 4/150
11/11 [==============================] - ETA: 0s - loss: 1.1498
Epoch 00004: loss improved from 1.15807 to 1.14976, saving model to model_1_weights_saved.hdf5
11/11 [==============================] - 1s 111ms/step - loss: 1.1498
Epoch 5/150
11/11 [=====================

Epoch 37/150
11/11 [==============================] - ETA: 0s - loss: 0.7667
Epoch 00037: loss did not improve from 0.76579
11/11 [==============================] - 1s 133ms/step - loss: 0.7667
Epoch 38/150
11/11 [==============================] - ETA: 0s - loss: 0.7656
Epoch 00038: loss improved from 0.76579 to 0.76563, saving model to model_1_weights_saved.hdf5
11/11 [==============================] - 1s 136ms/step - loss: 0.7656
Epoch 39/150
11/11 [==============================] - ETA: 0s - loss: 0.7773
Epoch 00039: loss did not improve from 0.76563
11/11 [==============================] - 1s 126ms/step - loss: 0.7773
Epoch 40/150
11/11 [==============================] - ETA: 0s - loss: 0.7250
Epoch 00040: loss improved from 0.76563 to 0.72498, saving model to model_1_weights_saved.hdf5
11/11 [==============================] - 2s 164ms/step - loss: 0.7250
Epoch 41/150
11/11 [==============================] - ETA: 0s - loss: 0.7215
Epoch 00041: loss improved from 0.72498 to 0.72150,

Epoch 75/150
11/11 [==============================] - ETA: 0s - loss: 0.4488
Epoch 00075: loss improved from 0.46133 to 0.44885, saving model to model_1_weights_saved.hdf5
11/11 [==============================] - 1s 129ms/step - loss: 0.4488
Epoch 76/150
11/11 [==============================] - ETA: 0s - loss: 0.4577
Epoch 00076: loss did not improve from 0.44885
11/11 [==============================] - 1s 117ms/step - loss: 0.4577
Epoch 77/150
11/11 [==============================] - ETA: 0s - loss: 0.4346
Epoch 00077: loss improved from 0.44885 to 0.43465, saving model to model_1_weights_saved.hdf5
11/11 [==============================] - 1s 133ms/step - loss: 0.4346
Epoch 78/150
11/11 [==============================] - ETA: 0s - loss: 0.4373
Epoch 00078: loss did not improve from 0.43465
11/11 [==============================] - 1s 122ms/step - loss: 0.4373
Epoch 79/150
11/11 [==============================] - ETA: 0s - loss: 0.4357
Epoch 00079: loss did not improve from 0.43465
11/1

Epoch 114/150
11/11 [==============================] - ETA: 0s - loss: 0.3395
Epoch 00114: loss did not improve from 0.29944
11/11 [==============================] - 1s 104ms/step - loss: 0.3395
Epoch 115/150
11/11 [==============================] - ETA: 0s - loss: 0.3319
Epoch 00115: loss did not improve from 0.29944
11/11 [==============================] - 1s 106ms/step - loss: 0.3319
Epoch 116/150
11/11 [==============================] - ETA: 0s - loss: 0.3161
Epoch 00116: loss did not improve from 0.29944
11/11 [==============================] - 1s 108ms/step - loss: 0.3161
Epoch 117/150
11/11 [==============================] - ETA: 0s - loss: 0.3156
Epoch 00117: loss did not improve from 0.29944
11/11 [==============================] - 1s 103ms/step - loss: 0.3156
Epoch 118/150
11/11 [==============================] - ETA: 0s - loss: 0.3092
Epoch 00118: loss did not improve from 0.29944
11/11 [==============================] - 1s 107ms/step - loss: 0.3092
Epoch 119/150
11/11 [====

In [90]:
filename = "model_1_weights_saved.hdf5" 
model.load_weights(filename) 
model.compile(loss='categorical_crossentropy', optimizer='adam') 

In [95]:
melody_pred = np.zeros((5))
X_pred = np.zeros((1,5,8))

melody_pred[0] = int(np.round(7*np.random.rand()))
melody_pred[1] = int(np.round(7*np.random.rand()))
melody_pred[2] = int(np.round(7*np.random.rand()))
melody_pred[3] = int(np.round(7*np.random.rand()))
melody_pred[4] = int(np.round(7*np.random.rand()))

for i in range(melody_pred.shape[0]):
    X_pred[0][i][int(melody_pred[i])] = 1
    
X_pred_012 = X_pred

In [96]:
final_midi = []
zeros = np.zeros((8))
for i in range(0,19):
    prediction = np.argmax(model.predict(X_pred, verbose = 0))
    
    final_midi.append(prediction)
    X_pred[0][0] = X_pred[0][1]
    X_pred[0][1] = X_pred[0][2]
    X_pred[0][2] = X_pred[0][3]
    X_pred[0][3] = X_pred[0][4]
    X_pred[0][4] = zeros
    X_pred[0][4][prediction] = 1

melody_start = []
for note in melody_pred:
    melody_start.append(int(note))
#melody_start
final_midi = melody_start + final_midi

In [97]:
to_pitch_dict = {0: 0, 1: 2, 2:3, 3:5, 4:7, 5:8, 6:10, 7:12}


def to_pitch(final_midi):
    tmp = []
    for note in final_midi:
        tmp.append(to_pitch_dict[note])
    return tmp

def shift_back(final_midi, lowest):
    tmp = []
    for note in final_midi:
        tmp.append(note + lowest)
    return tmp

final_shifted = shift_back(to_pitch(final_midi), 69)

In [98]:
export_midi = pm.PrettyMIDI('Dataset/1.mid')

for i in range(24):
    export_midi.instruments[0].notes[i].pitch = final_shifted[i]

export_midi.write('Output.mid')

In [100]:
X[0]

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.]])